In [1]:
import sys
import os
sys.path.append('../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('ggplot')
sns.set_palette("husl")
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

# Création des dossiers
os.makedirs("../models", exist_ok=True)
os.makedirs("../results", exist_ok=True)
os.makedirs("../reports", exist_ok=True)

print("✅ Configuration terminée")


✅ Configuration terminée


In [2]:
## 2. Chargement des Données
# %%
print("📥 Chargement des données...")
df = pd.read_parquet("../data/processed/model_data.parquet")
print(f"📊 Shape: {df.shape}")
print(f"🎯 Cible: delay_minutes")

📥 Chargement des données...
📊 Shape: (52453, 23107)
🎯 Cible: delay_minutes


In [3]:
# Aperçu
print("\n📋 Aperçu des données:")
print(df.head())
print("\n📈 Types de données:")
print(df.dtypes.value_counts())

# Distribution de la cible
print("\n📊 Statistiques de la variable cible (delay_minutes):")
print(f"• Moyenne: {df['delay_minutes'].mean():.2f} min")
print(f"• Médiane: {df['delay_minutes'].median():.2f} min")
print(f"• Std: {df['delay_minutes'].std():.2f} min")
print(f"• Min: {df['delay_minutes'].min():.2f} min")
print(f"• Max: {df['delay_minutes'].max():.2f} min")


📋 Aperçu des données:
   delay_minutes  hour  dayofweek  month  heure_pointe  incident_majeur  TransportType_Trains  TransportType_Tramways  Line_B  Line_C  Line_D  Line_E  Line_H  Line_J  Line_K  Line_L  Line_M1  Line_M10  Line_M11  Line_M12  Line_M13  Line_M14  Line_M2  Line_M3  Line_M3B  ...  IncidentCause_De 23h58 à 00h36 : Bagage oublié sur un quai 38 min Bagage oublié sur un quai  IncidentCause_De 23h58 à 00h41 : Bagage oublié dans un train 43 min Bagage oublié dans un train  IncidentCause_De 23h58 à 01h00 : Acte de malveillance 62 min Acte de malveillance  IncidentCause_De 23h58 à 01h15 : Incident d'exploitation 77 min Incident d'exploitation  IncidentCause_De 23h58 à 01h15 : Tramway en panne 77 min Tramway en panne  IncidentCause_De 23h59 à 00h00 : Malaise voyageur 1 min Malaise voyageur  IncidentCause_De 23h59 à 00h13 : Personnes sur les voies 13 min Personnes sur les voies  IncidentCause_De 23h59 à 00h45 : Incident (intervention conducteur) 46 min Incident (intervention cond

In [4]:
# Catégorisation pour classification
df['retard_categorie'] = pd.cut(df['delay_minutes'], 
                               bins=[-1, 0, 5, 15, 30, float('inf')],
                               labels=['ponctuel', 'petit_retard', 
                                      'moyen_retard', 'gros_retard', 
                                      'retard_majeur'])
df['retard_binaire'] = (df['delay_minutes'] > 5).astype(int)

print(f"\n📊 Distribution des catégories:")
print(df['retard_categorie'].value_counts())
print(f"\n🎯 Proportion de retards >5min: {df['retard_binaire'].mean()*100:.1f}%")


📊 Distribution des catégories:
retard_categorie
moyen_retard     19711
gros_retard      13043
retard_majeur    11217
petit_retard      8482
ponctuel             0
Name: count, dtype: int64

🎯 Proportion de retards >5min: 83.8%


In [5]:
## 3. Préparation des Données
# %%
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

print("🔧 Préparation des données...")

# Séparation features/target
X = df.drop(columns=['delay_minutes', 'retard_categorie', 'retard_binaire'])
y_reg = df['delay_minutes']
y_clf = df['retard_binaire']

print(f"📐 X shape: {X.shape}")
print(f"🎯 y_reg shape: {y_reg.shape}")
print(f"🏷️ y_clf shape: {y_clf.shape}")

🔧 Préparation des données...
📐 X shape: (52453, 23106)
🎯 y_reg shape: (52453,)
🏷️ y_clf shape: (52453,)


In [6]:
# Split train-test
X_train, X_test, y_train_reg, y_test_reg = train_test_split(
    X, y_reg, test_size=0.2, random_state=42, stratify=y_clf
)

X_train_clf, X_test_clf, y_train_clf, y_test_clf = train_test_split(
    X, y_clf, test_size=0.2, random_state=42, stratify=y_clf
)

print(f"\n📊 Split terminé:")
print(f"• Train (reg): {X_train.shape}")
print(f"• Test (reg): {X_test.shape}")
print(f"• Train (clf): {X_train_clf.shape}")
print(f"• Test (clf): {X_test_clf.shape}")



📊 Split terminé:
• Train (reg): (41962, 23106)
• Test (reg): (10491, 23106)
• Train (clf): (41962, 23106)
• Test (clf): (10491, 23106)


In [7]:
# Normalisation pour certains modèles
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

joblib.dump(scaler, "../models/scaler.pkl")
print("💾 Scaler sauvegardé")

MemoryError: Unable to allocate 7.22 GiB for an array with shape (23101, 41962) and data type float64